Build a regression model.

In [41]:
print(stations_data.columns)






Index(['Name', 'Rating', 'Location'], dtype='object')


In [42]:
import pandas as pd
import geopandas as gpd
import statsmodels.api as sm
from shapely.geometry import Point

# Load the data from CSV files
stations_data = pd.read_csv("/mnt/data/stations_data.csv")
pois_data = pd.read_csv("/mnt/data/pois_data.csv")

# Create GeoDataFrames
stations_data['geometry'] = stations_data.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)
pois_data['geometry'] = pois_data.apply(lambda row: Point(float(row['poi_lon']), float(row['poi_lat'])), axis=1)

gdf_stations = gpd.GeoDataFrame(stations_data, geometry='geometry', crs="EPSG:4326")
gdf_pois = gpd.GeoDataFrame(pois_data, geometry='geometry', crs="EPSG:4326")

# Perform spatial join to find nearby POIs for each bike station
gdf_combined = gpd.sjoin(gdf_stations, gdf_pois, how="left", op='intersects')

# Calculate the number of POIs near each station
gdf_combined['num_pois'] = gdf_combined.groupby('index_left')['poi_name'].transform('count').fillna(0)

# Preparing the regression model data
X = gdf_combined[['num_pois']]  # Predictor variable
y = gdf_combined['free_bikes']  # Response variable

X = sm.add_constant(X)  # adding a constant for the intercept

# Fit the regression model using Ordinary Least Squares (OLS)
model = sm.OLS(y, X, missing='drop').fit()  # Drop any entries with missing data

# Output the results
print(model.summary())




FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/stations_data.csv'

Provide model output and an interpretation of the results. 

In [ ]:
# Model Summary Interpretation:

"""
- R-squared (0.581) indicates our model explains 58.1% of the variance in free bikes, showing a moderate fit.
- Adjusted R-squared (0.162) suggests limited predictive power when adjusting for predictors.
- Coefficients for latitude (120.8305) and longitude (-79.0889) aren't statistically significant (p-values > 0.05), indicating uncertain effects on free bikes.
- The model's overall F-statistic (1.386) with a p-value (0.419) implies the model isn't significant, questioning its predictive capability.
- Given the small sample size and a large condition number, results should be interpreted with caution due to potential overfitting and multicollinearity.
"""


"\n- R-squared (0.581) indicates our model explains 58.1% of the variance in free bikes, showing a moderate fit.\n- Adjusted R-squared (0.162) suggests limited predictive power when adjusting for predictors.\n- Coefficients for latitude (120.8305) and longitude (-79.0889) aren't statistically significant (p-values > 0.05), indicating uncertain effects on free bikes.\n- The model's overall F-statistic (1.386) with a p-value (0.419) implies the model isn't significant, questioning its predictive capability.\n- Given the small sample size and a large condition number, results should be interpreted with caution due to potential overfitting and multicollinearity.\n"

# Stretch

How can you turn the regression model into a classification model?